# RAG Training Notebook for TA Chatbot

This notebook trains the complete RAG system:
1. **Retriever Training** - Fine-tune Sentence-BERT to find relevant documents
2. **Document Indexing** - Create embeddings for all documents
3. **Generator Training** - Fine-tune Mistral-7B with LoRA (coming soon)

**Expected Time:**
- Retriever: 30-60 minutes
- Indexing: 5-10 minutes
- Generator: 2-4 hours

**Requirements:** GPU runtime (Runtime → Change runtime type → GPU)


## Step 1: Setup and Mount Google Drive

Upload your project folder to Google Drive, then mount it here.


In [1]:
from google.colab import drive
import os
from pathlib import Path

# Mount Google Drive
drive.mount('/content/drive')

# Set project path (adjust if your folder is in a different location)
PROJECT_PATH = '/content/drive/MyDrive/TAChatBot'  # Change this to your actual path

# If you uploaded as a zip file, uncomment and adjust:
# !cd /content/drive/MyDrive && unzip -q TAChatBot.zip -d /content/
# PROJECT_PATH = '/content/TAChatBot'

# Change to project directory
os.chdir(PROJECT_PATH)
print(f"Working directory: {os.getcwd()}")

# Verify project structure
if os.path.exists('data/splits/train.json'):
    print("✅ Project files found!")
else:
    print("❌ Project files not found. Please check PROJECT_PATH.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Working directory: /content/drive/MyDrive/TAChatBot
✅ Project files found!


## Step 2: Install Dependencies

Install all required packages for training.


In [2]:
# Install required packages
%pip install -q torch transformers sentence-transformers peft bitsandbytes accelerate
%pip install -q chromadb rank-bm25 pandas numpy datasets scikit-learn tqdm

# Verify installation and GPU
import torch
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected! Please enable GPU: Runtime → Change runtime type → GPU")


✅ PyTorch version: 2.9.0+cu126
✅ CUDA available: True
✅ GPU: NVIDIA L4
✅ GPU Memory: 23.8 GB


## Step 3: Verify Data Files

Check that all data files are present and correct.


In [3]:
import json
from pathlib import Path

# Check data files
data_files = {
    'Train': 'data/splits/train.json',
    'Val': 'data/splits/val.json',
    'Test': 'data/splits/test.json',
    'Documents': 'data/splits/documents.json'
}

print("📊 Data Files Status:\n")
for name, path in data_files.items():
    file_path = Path(path)
    if file_path.exists():
        size_kb = file_path.stat().st_size / 1024
        with open(file_path, 'r') as f:
            data = json.load(f)
        print(f"✅ {name}: {path}")
        print(f"   Size: {size_kb:.1f} KB, Items: {len(data)}")
    else:
        print(f"❌ {name}: {path} - NOT FOUND")

# Show sample data
print("\n📄 Sample Training Data:")
with open('data/splits/train.json', 'r') as f:
    train_data = json.load(f)
    sample = train_data[0]
    print(f"   Question: {sample['question'][:80]}...")
    print(f"   Answer length: {len(sample['answer'])} chars")
    print(f"   Source: {sample['source']}")


📊 Data Files Status:

✅ Train: data/splits/train.json
   Size: 97.9 KB, Items: 109
✅ Val: data/splits/val.json
   Size: 11.3 KB, Items: 14
✅ Test: data/splits/test.json
   Size: 11.5 KB, Items: 14
✅ Documents: data/splits/documents.json
   Size: 623.0 KB, Items: 889

📄 Sample Training Data:
   Question: Frequent changes on HW2 checklist
I noticed that the HW2 checklist has been upda...
   Answer length: 955 chars
   Source: piazza


## Step 4: Train Retriever (Sentence-BERT)

Fine-tune Sentence-BERT to learn how to match questions to relevant documents.

**This will take 30-60 minutes.**


In [4]:
import sys
from pathlib import Path
import os

# Disable wandb (metrics will be printed directly)
os.environ['WANDB_DISABLED'] = 'true'

# Add src to path
sys.path.insert(0, str(Path('src')))

import json
import pandas as pd
from src.retriever import RetrieverTrainer

# Load training data
print("📖 Loading data splits...")
with open('data/splits/train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
with open('data/splits/val.json', 'r', encoding='utf-8') as f:
    val_data = json.load(f)

train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)

print(f"   Train: {len(train_df)} examples")
print(f"   Val: {len(val_df)} examples")

# Initialize trainer
print("\n🔧 Initializing RetrieverTrainer...")
print("   📊 WandB is disabled - metrics will be printed directly")
trainer = RetrieverTrainer(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    device=None  # Auto-detect (will use CUDA if available)
)

# Prepare training data with hard negatives
print("\n📊 Preparing training data with hard negatives...")
train_examples = trainer.prepare_training_data(
    train_df,
    num_hard_negatives=3  # Number of hard negatives per question
)

print(f"   Total training examples: {len(train_examples)}")


📖 Loading data splits...
   Train: 109 examples
   Val: 14 examples

🔧 Initializing RetrieverTrainer...
   📊 WandB is disabled - metrics will be printed directly
Loading model: sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model loaded on cuda

📊 Preparing training data with hard negatives...
Creating BM25 index for hard negative mining...
Preparing training examples with 3 hard negatives each...
Created 327 training examples
   Total training examples: 327


In [5]:
# Train the model
print("\n🏋️  Starting training...")
print("   This will take 30-60 minutes...")
print("   Metrics will be printed after each epoch (no WandB needed).\n")

output_dir = Path('models/retriever-finetuned')
output_dir.mkdir(parents=True, exist_ok=True)

# Train with validation evaluation after each epoch
# Returns training history with metrics for each epoch
training_history = trainer.train(
    train_examples,
    output_path=str(output_dir),
    epochs=5,  # Adjust based on your needs (3-5 is typical)
    batch_size=16,  # Adjust if you run out of memory (try 8 or 4)
    warmup_steps=100,
    val_df=val_df  # Pass validation data for epoch-by-epoch evaluation
)

print(f"\n✅ Training complete! Model saved to {output_dir}")
print(f"\n📊 You can see all metrics printed above - no WandB dashboard needed!")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



🏋️  Starting training...
   This will take 30-60 minutes...
   Metrics will be printed after each epoch (no WandB needed).


Training Configuration:
  Total examples: 327
  Batch size: 16
  Steps per epoch: 21
  Total epochs: 5
  Total steps: 105
  Warmup steps: 100
  Validation examples: 14

Starting Training...


────────────────────────────────────────────────────────────
Epoch 1/5
────────────────────────────────────────────────────────────
Training...


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss



Evaluating on validation set...

────────────────────────────────────────────────────────────
Epoch 1 Results:
────────────────────────────────────────────────────────────
  Precision@1: 0.5000
  Precision@3: 0.6429
  Precision@5: 0.7857
  MRR (Mean Reciprocal Rank): 0.6238
────────────────────────────────────────────────────────────
  ✅ New best model! (Precision@3: 0.6429)


...

Step,Training Loss


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



Evaluating on validation set...

────────────────────────────────────────────────────────────
Epoch 4 Results:
────────────────────────────────────────────────────────────
  Precision@1: 0.7143
  Precision@3: 0.9286
  Precision@5: 1.0000
  MRR (Mean Reciprocal Rank): 0.8119
────────────────────────────────────────────────────────────
  (Best Precision@3: 0.9286 at epoch 3)

Epoch 4/5 complete ✓

────────────────────────────────────────────────────────────
Epoch 5/5
────────────────────────────────────────────────────────────
Training...


Step,Training Loss



Evaluating on validation set...

────────────────────────────────────────────────────────────
Epoch 5 Results:
────────────────────────────────────────────────────────────
  Precision@1: 0.7857
  Precision@3: 0.8571
  Precision@5: 1.0000
  MRR (Mean Reciprocal Rank): 0.8536
────────────────────────────────────────────────────────────
  (Best Precision@3: 0.9286 at epoch 3)

Epoch 5/5 complete ✓

Training Complete!

📊 Training Summary:
  Best Precision@3: 0.9286 (Epoch 3)

  Training History:
    Epoch 1:
      precision@1: 0.5000
      precision@3: 0.6429
      precision@5: 0.7857
      mrr: 0.6238
    Epoch 2:
      precision@1: 0.5714
      precision@3: 0.7857
      precision@5: 0.8571
      mrr: 0.6845
    Epoch 3:
      precision@1: 0.6429
      precision@3: 0.9286
      precision@5: 1.0000
      mrr: 0.7643
    Epoch 4:
      precision@1: 0.7143
      precision@3: 0.9286
      precision@5: 1.0000
      mrr: 0.8119
    Epoch 5:
      precision@1: 0.7857
      precision@3: 0.8571
 

In [6]:
# Detailed evaluation on validation set (Precision@k, MRR)
# Note: Validation was already evaluated during training after each epoch
# This provides more detailed metrics (Precision@k, MRR)
print("\n📈 Running detailed evaluation on validation set...")
print("   (Note: Validation scores were already shown during training)")

metrics = trainer.evaluate(
    val_df,
    k_values=[1, 3, 5]
)

print("\n" + "="*60)
print("📊 Detailed Validation Results:")
print("="*60)
for metric, value in metrics.items():
    print(f"   {metric}: {value:.4f}")

# Check if metrics are good
print("\n" + "="*60)
if metrics.get('Precision@3', 0) > 0.5:
    print("✅ Good performance! Precision@3 > 0.5")
else:
    print("⚠️  Consider training for more epochs if Precision@3 < 0.5")
print("="*60)



📈 Running detailed evaluation on validation set...
   (Note: Validation scores were already shown during training)
Encoding questions and answers...
Precision@1: 0.7857
Precision@3: 0.8571
Precision@5: 1.0000
MRR: 0.8536

📊 Detailed Validation Results:
   Precision@1: 0.7857
   Precision@3: 0.8571
   Precision@5: 1.0000
   MRR: 0.8536

✅ Good performance! Precision@3 > 0.5


## Step 5: Index Documents

Create embeddings for all documents using the fine-tuned retriever.

**This will take 5-10 minutes.**


In [7]:
from src.retriever import Retriever
from src.utils import count_documents_by_type

# Load documents
print("📖 Loading documents...")
with open('data/splits/documents.json', 'r', encoding='utf-8') as f:
    documents = json.load(f)

# Extract text and metadata
texts = [doc['text'] for doc in documents]

# ✅ FIX: Extract ALL metadata fields (not just 4!)
metadatas = [
    {
        'source': doc.get('source', 'unknown'),
        'source_detail': doc.get('source_detail', ''),
        'doc_type': doc.get('doc_type', 'unknown'),
        'doc_id': doc.get('doc_id', ''),
        # ✅ ADD THESE CRITICAL FIELDS:
        'assignment_number': doc.get('assignment_number', ''),
        'section_title': doc.get('section_title', ''),
        'assignment': doc.get('assignment', ''),
        'post_id': doc.get('post_id', ''),
        'post_number': doc.get('post_number', ''),
        'lecture': doc.get('lecture', ''),
        'page_number': doc.get('page_number', ''),
        'tags': doc.get('tags', [])
    }
    for doc in documents
]

ids = [doc.get('doc_id', f'doc_{i}') for i, doc in enumerate(documents)]

# Use utility function for proper counting
counts = count_documents_by_type(documents)
print(f"   Loaded {counts['total']} documents")
print(f"   - Slides: {counts['slide']}")
print(f"   - Assignments: {counts['assignment']}")
print(f"   - Piazza: {counts['piazza_total']} (notes: {counts['piazza_note']}, Q&A: {counts['piazza_qa']})")

# Verify metadata extraction
assignment_docs = [m for m in metadatas if m.get('doc_type') == 'assignment']
print(f"\n✅ Metadata extraction verified:")
print(f"   Assignment docs with assignment_number: {sum(1 for m in assignment_docs if m.get('assignment_number'))}")

📖 Loading documents...
   Loaded 889 documents
   - Slides: 728
   - Assignments: 32
   - Piazza: 129 (notes: 25, Q&A: 104)

✅ Metadata extraction verified:
   Assignment docs with assignment_number: 32


In [8]:
# Load fine-tuned retriever
print("\n🔧 Loading fine-tuned retriever...")
retriever = Retriever('models/retriever-finetuned')

# Index documents WITH METADATA
print("\n📚 Indexing documents...")
retriever.index_documents(texts, metadatas=metadatas)

# Save document metadata
metadata_file = Path('models/retriever-finetuned/document_metadata.json')
with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump({
        'documents': [
            {
                'id': doc_id,
                'text_preview': text[:100] + '...' if len(text) > 100 else text,
                'metadata': meta
            }
            for text, meta, doc_id in zip(texts, metadatas, ids)
        ],
        'total_documents': len(texts)
    }, f, indent=2, ensure_ascii=False)

print(f"✅ Document metadata saved to {metadata_file}")



🔧 Loading fine-tuned retriever...
Loading retriever from models/retriever-finetuned

📚 Indexing documents...
Indexing 889 documents...


Batches:   0%|          | 0/28 [00:00<?, ?it/s]

✓ Documents indexed in memory
✅ Document metadata saved to models/retriever-finetuned/document_metadata.json


In [9]:
print("🔍 Checking indexed metadata in retriever...\n")

# Check total indexed
print(f"Total documents indexed: {len(retriever.metadatas)}")

# Count by type with proper Piazza handling
doc_types = {}
for meta in retriever.metadatas:
    dtype = meta.get('doc_type', 'MISSING')
    doc_types[dtype] = doc_types.get(dtype, 0) + 1

print("\nDocument types in retriever:")
for dtype, count in sorted(doc_types.items()):
    print(f"  {dtype}: {count}")

# Show combined Piazza count
piazza_note_count = doc_types.get('piazza_note', 0)
piazza_qa_count = doc_types.get('piazza_qa', 0)
piazza_total = piazza_note_count + piazza_qa_count
print(f"\n  Piazza total: {piazza_total} (notes: {piazza_note_count}, Q&A: {piazza_qa_count})")

# Check assignment documents specifically
assignment_metas = [m for m in retriever.metadatas if m.get('doc_type') == 'assignment']
print(f"\nAssignment documents indexed: {len(assignment_metas)}")

if assignment_metas:
    print("\nFirst 3 assignment metadatas in retriever:")
    for i, meta in enumerate(assignment_metas[:3], 1):
        print(f"\n{i}. {meta.get('source', 'No source')}")
        print(f"   doc_type: {meta.get('doc_type', 'MISSING')}")
        print(f"   assignment_number: {meta.get('assignment_number', 'MISSING')} (type: {type(meta.get('assignment_number')).__name__})")
        print(f"   section_title: {meta.get('section_title', 'MISSING')}")
else:
    print("\n❌ NO ASSIGNMENT METADATA IN RETRIEVER!")
    print("   This means metadata wasn't passed during indexing!")
    print("   Check Cell 13: should have metadatas=metadatas parameter")

🔍 Checking indexed metadata in retriever...

Total documents indexed: 889

Document types in retriever:
  assignment: 32
  piazza_note: 25
  piazza_qa: 104
  slide: 728

  Piazza total: 129 (notes: 25, Q&A: 104)

Assignment documents indexed: 32

First 3 assignment metadatas in retriever:

1. Assignment 0 - Assignment Overview and Requirements
   doc_type: assignment
   assignment_number: 0 (type: str)
   section_title: Assignment Overview and Requirements

2. Assignment 0 - Assignment Overview and Requirements
   doc_type: assignment
   assignment_number: 0 (type: str)
   section_title: Assignment Overview and Requirements

3. Assignment 0 - Assignment Overview and Requirements
   doc_type: assignment
   assignment_number: 0 (type: str)
   section_title: Assignment Overview and Requirements


In [10]:
# Test retrieval with comprehensive test cases
print("\n🧪 Testing retrieval with diverse queries...")
print("=" * 80)

# Comprehensive test queries covering different scenarios
test_queries = [
    # ========== Assignment-Specific Queries ==========
    ("Assignment Queries", [
        "What are the requirements for assignment 1?",
        "What are the requirements for assignment 2?",
        "How do I submit assignment 0?",
        "What is the due date for assignment 1?",
        "Explain problem 1 in assignment 1",
        "What is problem 2 in assignment 2?",
        "How to implement backpropagation for assignment 1 problem 1?",
        "What datasets are used in assignment 2?",
    ]),

    # ========== Lecture/Concept Queries ==========
    ("Lecture & Concept Queries", [
        "How does backpropagation work?",
        "Explain neural networks",
        "What is gradient descent?",
        "What are convolutional neural networks?",
        "Explain the ReLU activation function",
        "What is overfitting in machine learning?",
    ]),

    # ========== Piazza/Discussion Queries ==========
    ("Piazza Discussion Queries", [
        "Can I use pretrained models for the project?",
        "What GPU resources are available?",
        "Is late submission allowed?",
    ]),
]

def format_source(metadata):
    """Format source information based on document type"""
    doc_type = metadata.get('doc_type', 'unknown')

    if doc_type == 'slide':
        # For slides: show lecture name and page
        source = metadata.get('source', '')
        source_detail = metadata.get('source_detail', '')
        # Extract filename and page from source_detail
        if ',' in source_detail:
            filename, page_info = source_detail.split(',', 1)
            return f"📊 {source} ({filename.strip()}, {page_info.strip()})"
        return f"📊 {source}"

    elif doc_type == 'assignment':
        # For assignments: show assignment name and section
        source_detail = metadata.get('source_detail', metadata.get('source', ''))
        assignment_num = metadata.get('assignment_number', '?')
        section = metadata.get('section_title', '')
        return f"📝 Assignment {assignment_num}: {section}"

    elif doc_type == 'piazza':
        # For Piazza: show post info
        source = metadata.get('source', 'Piazza')
        return f"💬 {source}"

    else:
        return f"📄 {metadata.get('source', 'Unknown')}"

# Run tests by category
for category, queries in test_queries:
    print(f"\n{'='*80}")
    print(f"📂 {category}")
    print(f"{'='*80}")

    for query in queries:
        print(f"\n📝 Query: '{query}'")

        # Retrieve 5 documents with smart ranking
        results = retriever.retrieve(query, top_k=5, return_metadata=True, use_smart_ranking=True)
        print(f"   Retrieved {len(results)} documents:")

        for i, (doc, score, metadata) in enumerate(results, 1):
            source_display = format_source(metadata)
            print(f"\n   {i}. Score: {score:.4f}")
            print(f"      Source: {source_display}")
            print(f"      Preview: {doc[:100]}...")

        print()  # Extra line between queries

print("\n" + "="*80)
print("✅ Retrieval test complete!")
print("="*80)



🧪 Testing retrieval with diverse queries...

📂 Assignment Queries

📝 Query: 'What are the requirements for assignment 1?'
   Retrieved 18 documents:

   1. Score: 0.9555
      Source: 📝 Assignment 1: Assignment Overview and Requirements
      Preview: # ELEC 576 / COMP 576 – Fall 2025  
## Assignment 1

**Due: Oct 7, 2025, 11:59 p.m. via Canvas**

--...

   2. Score: 0.9211
      Source: 📝 Assignment 1: Problem 2: Training a Simple Deep Convolutional Network on MNIST
      Preview: ---  # LLM Policy  **LLMs (ChatGPT, Copilot, etc.) are *not permitted* for coding in Assignment 1.**...

   3. Score: 0.7986
      Source: 📝 Assignment 1: Problem 2: Training a Simple Deep Convolutional Network on MNIST
      Preview: ---  ## c) More Experiments  Try different:  - Activations:   - tanh, sigmoid, leaky-ReLU, MaxOut   ...

   4. Score: 0.5785
      Source: 📝 Assignment 1: Problem 1: Backpropagation in a Simple Neural Network
      Preview: ---  ## c) Build the 3-Layer Network  Network structu

## Step 6: Download Models

Download the trained retriever model to your local machine.


In [11]:
from google.colab import files
import shutil

# Create zip file
model_dir = Path('models/retriever-finetuned')
if model_dir.exists():
    zip_path = 'retriever-finetuned.zip'
    shutil.make_archive('retriever-finetuned', 'zip', model_dir)

    print(f"✅ Created {zip_path}")
    print("📥 Downloading...")
    files.download(zip_path)
    print("✅ Download complete!")
else:
    print("❌ Model directory not found. Please train the model first.")


✅ Created retriever-finetuned.zip
📥 Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download complete!


---

# Part 2: Generator Training (Qwen 2.5 14B + LoRA)

Now that we have a trained retriever, let's train the generator to produce answers from retrieved context.

**What we'll do:**
1. Prepare training data with retrieved context
2. Load **Qwen 2.5 14B Instruct** with 4-bit quantization (upgraded from Mistral-7B!)
3. Configure LoRA for efficient fine-tuning  
4. Train the generator on Q&A pairs with retrieved context
5. Test the full RAG pipeline

**Why Qwen 2.5 14B?**
- 🚀 Excellent instruction following & reasoning (comparable to Llama 3.1 13B)
- ✅ **No authentication needed** - Works immediately!
- 💪 Only ~9GB in 4-bit (fits easily in 20GB GPU)
- 🎯 Superior at synthesizing answers from context
- ⚡ Same training time as smaller models

**Model Options Available:**
- **Qwen 2.5 14B** (default) - No auth, excellent quality ⭐
- Llama 3.1 13B - Requires HF auth, excellent quality
- Mistral-7B v0.3 - No auth, good quality, fastest

**Expected time:** 2-4 hours
**GPU required:** 15-20GB VRAM (tested on L4/A100)


## Step 8: Prepare Generator Training Data

Create training data with retrieved context for each question.


In [12]:
print("📋 Preparing generator training data with retrieved context...\n")

# For each training question, retrieve relevant context
def prepare_rag_training_data(train_df, val_df, retriever, top_k=3):
    """
    For each Q&A pair, retrieve relevant context and format for generator training
    """
    def format_with_context(row, retriever, top_k):
        question = row['question']
        answer = row['answer']

        # Retrieve relevant documents
        results = retriever.retrieve(question, top_k=top_k, return_metadata=True, use_smart_ranking=True)

        # Format context
        context_parts = []
        for i, (doc, score, metadata) in enumerate(results, 1):
            source = metadata.get('source', 'Unknown')
            context_parts.append(f"[{i}] {source}:\\n{doc[:500]}")  # Limit each doc to 500 chars

        context = "\\n\\n".join(context_parts)

        # Create prompt using Qwen 2.5 chat template format (manually, since tokenizer not loaded yet)
        # This matches what tokenizer.apply_chat_template() produces for Qwen
        system_message = "You are a teaching assistant for a deep learning course. Answer the question using ONLY the provided context. Be concise and accurate. Do NOT make up URLs, links, or references not in the context."

        user_message = f"""Context:
{context}

Question: {question}"""

        # Qwen 2.5 chat template format
        prompt = f"""<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{user_message}<|im_end|>
<|im_start|>assistant
"""

        return {
            'prompt': prompt,
            'answer': answer,
            'question': question,
            'context': context
        }

    print("Processing training data...")
    train_examples = []
    for _, row in train_df.iterrows():
        train_examples.append(format_with_context(row, retriever, top_k))

    print(f"   Prepared {len(train_examples)} training examples")

    print("Processing validation data...")
    val_examples = []
    for _, row in val_df.iterrows():
        val_examples.append(format_with_context(row, retriever, top_k))

    print(f"   Prepared {len(val_examples)} validation examples")

    return train_examples, val_examples

# Prepare data
train_examples, val_examples = prepare_rag_training_data(train_df, val_df, retriever, top_k=3)

# Show example
print("\\n" + "="*80)
print("📄 Example training instance:")
print("="*80)
example = train_examples[0]
print(f"Question: {example['question'][:100]}...")
print(f"\\nPrompt length: {len(example['prompt'])} chars")
print(f"Answer length: {len(example['answer'])} chars")
print(f"\\nPrompt preview:")
print(example['prompt'][:300] + "...")
print("\\n" + "="*80)


📋 Preparing generator training data with retrieved context...

Processing training data...
   Prepared 109 training examples
Processing validation data...
   Prepared 14 validation examples
\n================================================================================
📄 Example training instance:
Question: Frequent changes on HW2 checklist
I noticed that the HW2 checklist has been updated multiple times, ...
\nPrompt length: 2382 chars
Answer length: 955 chars
\nPrompt preview:
<|im_start|>system
You are a teaching assistant for a deep learning course. Answer the question using ONLY the provided context. Be concise and accurate. Do NOT make up URLs, links, or references not in the context.<|im_end|>
<|im_start|>user
Context:
[1] Piazza Q&A:\nQ: Frequent changes on HW2 chec...
\n================================================================================


## Step 9: Load Generator Model with LoRA Configuration

Load the base model with 4-bit quantization and configure LoRA for efficient fine-tuning.

**Model Options:**
- **Option 1: Llama 3.1 13B** (requires HuggingFace auth) - Best quality
- **Option 2: Qwen 2.5 14B** (no auth needed) - Excellent quality, easier setup
- **Option 3: Mistral-7B-Instruct-v0.3** (no auth needed) - Good quality, fastest

**Why upgrade from Mistral-7B?**
- 🚀 Much better instruction following
- 💪 Only ~8-9GB in 4-bit (fits easily in 20GB GPU)
- 🎯 Excellent at synthesizing from retrieved context
- ⚡ Same training time as 7B models


### Step 9a: HuggingFace Authentication (Only if using Llama)

**⚠️ Skip this cell if using Qwen 2.5 14B or Mistral (no auth needed)**

If you want to use Llama 3.1 13B:
1. Get a HuggingFace token: https://huggingface.co/settings/tokens
2. Request access to Llama: https://huggingface.co/meta-llama/Meta-Llama-3.1-13B-Instruct
3. Paste your token below


In [13]:
# 🔐 HuggingFace Authentication (ONLY if using Llama 3.1 13B)
# Skip this cell if using Qwen 2.5 14B or Mistral

USE_LLAMA = False  # Set to True if you have Llama access

if USE_LLAMA:
    from huggingface_hub import login

    # Option 1: Use token from environment variable (recommended)
    # Set HF_TOKEN environment variable in Colab secrets

    # Option 2: Paste token directly (less secure, but works)
    # token = "hf_your_token_here"  # ⚠️ Replace with your token
    # login(token=token)

    # Option 3: Interactive login
    login()  # Will prompt for token

    print("✅ HuggingFace authenticated!")
else:
    print("⏭️  Skipping authentication (using Qwen or Mistral)")


⏭️  Skipping authentication (using Qwen or Mistral)


In [14]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import torch

# 🎯 MODEL SELECTION - Choose one:
# Option 1: Qwen 2.5 14B (Recommended - No auth needed, excellent quality)
MODEL_CHOICE = "qwen"  # Options: "qwen", "llama", "mistral"

if MODEL_CHOICE == "qwen":
    model_name = "Qwen/Qwen2.5-14B-Instruct"
    print("🔧 Loading Qwen 2.5 14B Instruct (no auth needed)...\\n")
    print("   ✅ Excellent quality, easy setup!")
elif MODEL_CHOICE == "llama":
    model_name = "meta-llama/Meta-Llama-3.1-13B-Instruct"
    print("🔧 Loading Llama 3.1 13B Instruct (requires auth)...\\n")
    print("   ⚠️  Make sure you authenticated in previous cell!")
elif MODEL_CHOICE == "mistral":
    model_name = "mistralai/Mistral-7B-Instruct-v0.3"
    print("🔧 Loading Mistral-7B-Instruct-v0.3 (no auth needed)...\\n")
    print("   ✅ Good quality, fastest option")
else:
    raise ValueError(f"Unknown MODEL_CHOICE: {MODEL_CHOICE}")

print(f"   Model: {model_name}")
print(f"   This will take 2-3 minutes to download (~8-9GB)...\\n")

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading {model_name}...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Model loaded successfully!")
print(f"   Model memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")

if MODEL_CHOICE == "qwen":
    print(f"   🚀 Qwen 2.5 14B: Excellent quality, great instruction following!")
elif MODEL_CHOICE == "llama":
    print(f"   🚀 Llama 3.1 13B: ~30-40% better quality than 7B models!")
elif MODEL_CHOICE == "mistral":
    print(f"   ✅ Mistral-7B v0.3: Good quality, reliable performance!")

# Prepare model for k-bit training
print("\\n🔧 Preparing model for LoRA fine-tuning...")
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,                    # LoRA rank
    lora_alpha=16,          # LoRA alpha (scaling factor)
    target_modules=[        # Which layers to apply LoRA to
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\\n✅ LoRA configured!")
print(f"   Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"   Total parameters: {total_params:,}")


🔧 Loading Qwen 2.5 14B Instruct (no auth needed)...\n
   ✅ Excellent quality, easy setup!
   Model: Qwen/Qwen2.5-14B-Instruct
   This will take 2-3 minutes to download (~8-9GB)...\n
Loading Qwen/Qwen2.5-14B-Instruct...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

✅ Model loaded successfully!
   Model memory footprint: 9.72 GB
   🚀 Qwen 2.5 14B: Excellent quality, great instruction following!
\n🔧 Preparing model for LoRA fine-tuning...
\n✅ LoRA configured!
   Trainable parameters: 34,406,400 (0.42%)
   Total parameters: 8,198,411,264


## Step 10: Train Generator

Fine-tune the generator on Q&A pairs with retrieved context.

**This will take 2-4 hours depending on your GPU.**


## Step 10.5: Simple Generation Test (Without Retrieval)

Test if the model can generate coherent text on a simple prompt without RAG context.


In [15]:
# Simple test WITH chat template to verify model works
model.eval()

messages = [
    {"role": "system", "content": "You are a teaching assistant for a deep learning course."},
    {"role": "user", "content": "What is deep learning?"}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print("Testing model with proper chat template (before fine-tuning)...")
print(f"Prompt template applied: Qwen format")
print("-" * 80)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
    )

full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
# Extract answer after prompt
answer = full_output[len(prompt):].strip() if len(full_output) > len(prompt) else full_output

print(f"Generated Answer: '{answer}'")
print("-" * 80)

if len(answer) > 20:
    print("✅ Model is generating output (quality will improve after fine-tuning)")
else:
    print("⚠️ Model output is very short. This is normal before fine-tuning.")


Testing model with proper chat template (before fine-tuning)...
Prompt template applied: Qwen format
--------------------------------------------------------------------------------
Generated Answer: 'ves the use of artificial neural networks to model and solve complex problems. These models, often referred to as "deep" because they have multiple layers between input and output nodes (unlike simpler models which may only contain one layer), can learn hierarchical representations from raw data such as images, sound, text or time series.

Key aspects of Deep Learning include:

1. **Neural Networks**: Inspired by biological neurons in human brains, these networks consist of interconnected'
--------------------------------------------------------------------------------
✅ Model is generating output (quality will improve after fine-tuning)


In [16]:
print("🏋️  Preparing datasets for training...\\n")

# Tokenize function with PROPER LABEL MASKING
def tokenize_function(examples):
    """
    Tokenize and create labels that ONLY train on the answer part, not the prompt.
    This prevents the model from learning to repeat "Question:" patterns.
    """
    model_inputs = {"input_ids": [], "attention_mask": [], "labels": []}

    for prompt, answer in zip(examples['prompt'], examples['answer']):
        # Tokenize prompt and answer separately
        prompt_ids = tokenizer(prompt, add_special_tokens=True, truncation=True, max_length=1536)["input_ids"]
        answer_ids = tokenizer(answer + tokenizer.eos_token, add_special_tokens=False, truncation=True, max_length=512)["input_ids"]

        # Concatenate
        input_ids = prompt_ids + answer_ids

        # Create labels: -100 for prompt (ignored in loss), actual IDs for answer
        labels = [-100] * len(prompt_ids) + answer_ids

        # Pad or truncate to max_length
        max_length = 2048
        if len(input_ids) < max_length:
            # Pad
            padding_length = max_length - len(input_ids)
            input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
            labels = labels + [-100] * padding_length
            attention_mask = [1] * (len(input_ids) - padding_length) + [0] * padding_length
        else:
            # Truncate
            input_ids = input_ids[:max_length]
            labels = labels[:max_length]
            attention_mask = [1] * max_length

        model_inputs["input_ids"].append(input_ids)
        model_inputs["attention_mask"].append(attention_mask)
        model_inputs["labels"].append(labels)

    return model_inputs

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_dict({
    'prompt': [ex['prompt'] for ex in train_examples],
    'answer': [ex['answer'] for ex in train_examples]
})

val_dataset = Dataset.from_dict({
    'prompt': [ex['prompt'] for ex in val_examples],
    'answer': [ex['answer'] for ex in val_examples]
})

# Tokenize
print("Tokenizing training data...")
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['prompt', 'answer'])

print("Tokenizing validation data...")
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=['prompt', 'answer'])

print(f"\\n✅ Datasets ready!")
print(f"   Train: {len(train_dataset)} examples")
print(f"   Val: {len(val_dataset)} examples")

# Configure training
output_dir = "models/generator-finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,      # Small batch due to memory
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,      # Increased to 8 (was 4) - larger effective batch = more stable

    # 🔧 ULTRA-STABLE: Even more conservative learning rate
    learning_rate=5e-5,                 # Further reduced from 1e-4 (maximum stability)
    lr_scheduler_type="linear",         # Changed from cosine (smoother, more predictable decay)
    warmup_ratio=0.15,                  # Increased from 0.1 (even gentler start)

    max_grad_norm=1,

    # 🔧 IMPROVED LOGGING & EVALUATION
    logging_steps=5,                    # Log every 5 steps
    logging_first_step=True,            # See first step immediately
    eval_strategy="steps",
    eval_steps=10,                      # Evaluate every 10 steps

    save_strategy="steps",
    save_steps=50,                      # Save checkpoints
    save_total_limit=3,                 # Keep 3 best checkpoints
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",                   # Disable wandb
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    # Additional stability
    gradient_checkpointing=True,        # Save memory
    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataloader_pin_memory=False,        # Better stability in some cases
    remove_unused_columns=False,        # Prevent unexpected behavior
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train
print("\\n" + "="*80)
print("🏋️  Starting generator training...")
print("="*80)
print("\\n⚡ ULTRA-STABLE training configuration:")
print("   - Learning rate: 5e-5 (very conservative, maximum stability)")
print("   - Gradient clipping: max_norm=0.5 (strict prevention of spikes)")
print("   - Gradient accumulation: 8 steps (effective batch size = 8)")
print("   - Warmup: 15% (gentle start, smooth ramp-up)")
print("   - LR scheduler: linear (predictable, smooth decay)")
print("   - Logging every 5 steps (immediate feedback)")
print("   - Evaluation every 10 steps (~3x per epoch)")
print("   - Proper label masking (only trains on answers, not prompts)")
print("\\n✅ Expected behavior: SMOOTH loss decrease")
print("   Step 10:  2.5 → Step 20: 2.3 → Step 30: 2.1 → Step 40: 1.9")
print("   Small jumps (±0.1-0.2) are OK, but NO large spikes!")
print("\\n⏱️  This will take 2-4 hours.\\n")

trainer.train()

# Save final model
print("\\n" + "="*80)
print("💾 Saving final model...")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ Model saved to {output_dir}")
print("="*80)


🏋️  Preparing datasets for training...\n
Tokenizing training data...


Map:   0%|          | 0/109 [00:00<?, ? examples/s]

Tokenizing validation data...


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

\n✅ Datasets ready!
   Train: 109 examples
   Val: 14 examples
\n================================================================================
🏋️  Starting generator training...
\n⚡ ULTRA-STABLE training configuration:
   - Learning rate: 5e-5 (very conservative, maximum stability)
   - Gradient clipping: max_norm=0.5 (strict prevention of spikes)
   - Gradient accumulation: 8 steps (effective batch size = 8)
   - Warmup: 15% (gentle start, smooth ramp-up)
   - LR scheduler: linear (predictable, smooth decay)
   - Logging every 5 steps (immediate feedback)
   - Evaluation every 10 steps (~3x per epoch)
   - Proper label masking (only trains on answers, not prompts)
\n✅ Expected behavior: SMOOTH loss decrease
   Step 10:  2.5 → Step 20: 2.3 → Step 30: 2.1 → Step 40: 1.9
   Small jumps (±0.1-0.2) are OK, but NO large spikes!
\n⏱️  This will take 2-4 hours.\n


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
10,0.988300,1.965293
20,0.531600,1.123032
30,0.683600,1.009519
40,0.566800,0.980585


\n================================================================================
💾 Saving final model...
✅ Model saved to models/generator-finetuned


In [17]:
# Check label masking
sample = train_dataset[0]
print("First 20 labels:", sample["labels"][:20])
print("Last 20 labels:", sample["labels"][-20:])
print(f"Number of -100 tokens: {sum(1 for x in sample['labels'] if x == -100)}")
print(f"Total labels: {len(sample['labels'])}")

First 20 labels: [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Last 20 labels: [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Number of -100 tokens: 1869
Total labels: 2048


## Step 11: Test Full RAG Pipeline

Test the complete system: Retrieval + Generation


In [18]:
print("🧪 Testing Full RAG Pipeline\\n")
print("="*80)

# ⚡ IMPORTANT: Put model in evaluation mode
model.eval()
print("✅ Model set to evaluation mode\\n")

# RAG function
def rag_answer(question, retriever, model, tokenizer, top_k=3, max_new_tokens=768):
    """
    Complete RAG pipeline: Retrieve + Generate

    Note: max_new_tokens=768 (was 512) to allow for longer, more detailed answers.
    Combined with min_new_tokens=50 and length_penalty=1.1, this encourages
    comprehensive responses rather than short snippets.
    """
    # Step 1: Retrieve relevant documents
    results = retriever.retrieve(question, top_k=top_k, return_metadata=True, use_smart_ranking=True)

    # Step 2: Format context (reduce doc length to avoid truncation)
    context_parts = []
    for i, (doc, score, metadata) in enumerate(results, 1):
        source = metadata.get('source', 'Unknown')
        # Reduce from 500 to 300 chars per doc to fit more context
        context_parts.append(f"[{i}] {source}:\\n{doc[:300]}")

    context = "\\n\\n".join(context_parts)

    # Step 3: Create prompt using chat template
    system_message = "You are a teaching assistant for a deep learning course. Answer the question using ONLY the provided context. Be concise and accurate. IMPORTANT: Do NOT make up URLs, links, or references that are not explicitly in the context. If you don't know something, say so."

    user_message = f"""Context:
{context}

Question: {question}"""

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    # Apply Qwen's chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Step 4: Generate answer
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

        # 🔧 IMPROVED GENERATION PARAMETERS
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            min_new_tokens=50,               # ✅ Ensure minimum answer length (50 tokens ≈ 100-150 chars)
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,

            # 🚫 PREVENT REPETITION
            repetition_penalty=1.2,          # Penalize repeated tokens
            no_repeat_ngram_size=3,          # Don't repeat 3-grams

            # 📏 ENCOURAGE LONGER ANSWERS
            length_penalty=1.1,              # ✅ Slight preference for longer answers (1.0 = neutral, >1.0 = longer)

            # Better stopping
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode ONLY the new tokens (not the input prompt)
    # This avoids tokenization mismatch issues
    input_length = inputs["input_ids"].shape[1]
    generated_tokens = outputs[0][input_length:]
    answer = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    return {
        'question': question,
        'answer': answer,
        'context': context,
        'retrieved_docs': results
    }

# Test queries
test_queries = [
    "What are the requirements for assignment 1?",
    "How does backpropagation work?",
    "What is the due date for assignment 2?",
    "Explain convolutional neural networks",
    "What GPU resources are available?"
]

print("\\n🔍 Testing RAG pipeline with sample questions...\\n")

for i, query in enumerate(test_queries, 1):
    print("="*80)
    print(f"Test {i}/{len(test_queries)}")
    print("="*80)
    print(f"\\n📝 Question: {query}")

    # Get RAG answer
    result = rag_answer(query, retriever, model, tokenizer)

    print(f"\\n🤖 Generated Answer:")
    print(result['answer'])

    print(f"\\n📚 Retrieved Documents:")
    for j, (doc, score, metadata) in enumerate(result['retrieved_docs'], 1):
        source = metadata.get('source', 'Unknown')
        print(f"   {j}. {source} (score: {score:.4f})")

    print()

print("="*80)
print("✅ RAG Pipeline Testing Complete!")
print("="*80)


The following generation flags are not valid and may be ignored: ['length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧪 Testing Full RAG Pipeline\n
✅ Model set to evaluation mode\n
\n🔍 Testing RAG pipeline with sample questions...\n
Test 1/5
\n📝 Question: What are the requirements for assignment 1?
\n🤖 Generated Answer:
Assignment 1 has several components and tasks related to implementing backpropogation and building simple neural networks from scratch. In addition, it involves applying these techniques to real-world datasets such as MNIST digits data. Students should train models on these datasets, perform experiments by varying model architecture parameters like hidden units, weight initialization methods, activation types and optimization settings. 

They then analyze results through visualizations including decision boundaries and tensorboard graphs tracking metrics during epochs. Lastly they write reports summarizing findings into markdown files converted to PDF format alongside packaged Python scripts. Specific details can be found at <https://www.dropbox.com/scl/f/iJgKtjFZCkVdXmUOuEiMw/shared/p

## Step 12: Download Trained Models

Download both retriever and generator models to your local machine.


In [19]:
from google.colab import files
import shutil
from pathlib import Path

print("📦 Preparing models for download...\\n")

# Create zip files for both models
models_to_download = [
    ('models/retriever-finetuned', 'retriever-finetuned.zip'),
    ('models/generator-finetuned', 'generator-finetuned.zip')
]

for model_dir, zip_name in models_to_download:
    model_path = Path(model_dir)
    if model_path.exists():
        print(f"📦 Creating {zip_name}...")
        shutil.make_archive(zip_name.replace('.zip', ''), 'zip', model_path)
        print(f"   ✅ {zip_name} created ({Path(zip_name).stat().st_size / 1e6:.1f} MB)")
    else:
        print(f"   ❌ {model_dir} not found - skipping")

print("\\n📥 Downloading models...")
print("(This may take a few minutes depending on model size)\\n")

for model_dir, zip_name in models_to_download:
    if Path(zip_name).exists():
        print(f"Downloading {zip_name}...")
        files.download(zip_name)
        print(f"   ✅ {zip_name} downloaded")

print("\\n✅ All models downloaded!")
print("\\n" + "="*80)
print("🎉 Training Complete!")
print("="*80)
print("\\nYou now have:")
print("  ✅ Fine-tuned retriever (Sentence-BERT)")
print("  ✅ Fine-tuned generator (Mistral-7B + LoRA)")
print("  ✅ Full RAG pipeline tested")
print("\\nNext steps:")
print("  1. Extract the zip files locally")
print("  2. Use the models in your production environment")
print("  3. Integrate into a web app or API")
print("="*80)


📦 Preparing models for download...\n
📦 Creating retriever-finetuned.zip...
   ✅ retriever-finetuned.zip created (83.7 MB)
📦 Creating generator-finetuned.zip...
   ✅ generator-finetuned.zip created (700.5 MB)
\n📥 Downloading models...
(This may take a few minutes depending on model size)\n


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   ✅ retriever-finetuned.zip downloaded


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   ✅ generator-finetuned.zip downloaded
\n✅ All models downloaded!
\n================================================================================
🎉 Training Complete!
\nYou now have:
  ✅ Fine-tuned retriever (Sentence-BERT)
  ✅ Fine-tuned generator (Mistral-7B + LoRA)
  ✅ Full RAG pipeline tested
\nNext steps:
  1. Extract the zip files locally
  2. Use the models in your production environment
  3. Integrate into a web app or API
